In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import numpy as np
import pandas as pd
import cv2
from PIL import Image

In [18]:
input_size = 28
num_classes = 10
num_epochs = 20
batch_size = 500

train_dataset = torchvision.datasets.CIFAR10(root='data',
                              train=True,
                              transform=torchvision.transforms.ToTensor())

test_dataset = torchvision.datasets.CIFAR10(root='data',
                             train=False,
                             transform=torchvision.transforms.ToTensor())

In [16]:
#构建 mini-batch
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [12]:
#构建 网络结构
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=3,
                                             out_channels=32,
                                             kernel_size=5,
                                             stride=1,
                                             padding=2),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2),)
        self.conv2 = nn.Sequential(nn.Conv2d(32, 64, 5, 1, 2),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2),)
        self.out = nn.Sequential(nn.Flatten(),
                                 nn.Linear(64 * 8 * 8, 256),
                                 nn.Dropout(),
                                 nn.ReLU(),
                                 nn.Linear(256, 10),)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        output = self.out(x)
        return output

In [13]:
#定义准确率作为评估标准
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]            #torch.max()[0]返回最大值 torch.max()[1]返回最大值位置，既预测值
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)                          #这里返回的是正确个数与总个数组成的元组

In [19]:
net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
net.cuda()
test_right = []
train_loss = []

for epoch in range(num_epochs):
    train_rights = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx % 50 == 0:
            net.eval()
            val_rights = []
            
            for (data, target) in test_loader:
                data, target = data.cuda(), target.cuda()
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
            
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
            test_right.append((100*val_r[0]/val_r[1]).cpu().numpy())
            train_loss.append(loss.data.cpu().numpy())

            print(f'当前epoch：{epoch} [{batch_idx*batch_size}/{len(train_loader.dataset)}]   损失：{loss.data:.6f}  训练集准确率：{100*train_r[0]/train_r[1]:.2f}%   测试集准确率：{100*val_r[0]/val_r[1]:.2f}%')

当前epoch：0 [0/50000]   损失：2.303518  训练集准确率：10.40%   测试集准确率：10.05%
当前epoch：0 [25000/50000]   损失：1.681545  训练集准确率：29.01%   测试集准确率：41.61%
当前epoch：1 [0/50000]   损失：1.578475  训练集准确率：44.20%   测试集准确率：46.85%
当前epoch：1 [25000/50000]   损失：1.383550  训练集准确率：45.74%   测试集准确率：49.11%
当前epoch：2 [0/50000]   损失：1.342069  训练集准确率：54.00%   测试集准确率：54.21%
当前epoch：2 [25000/50000]   损失：1.280360  训练集准确率：52.98%   测试集准确率：57.26%
当前epoch：3 [0/50000]   损失：1.287874  训练集准确率：57.20%   测试集准确率：58.21%
当前epoch：3 [25000/50000]   损失：1.155610  训练集准确率：56.66%   测试集准确率：59.33%
当前epoch：4 [0/50000]   损失：1.166305  训练集准确率：59.60%   测试集准确率：60.07%
当前epoch：4 [25000/50000]   损失：1.112506  训练集准确率：59.55%   测试集准确率：62.19%
当前epoch：5 [0/50000]   损失：1.086016  训练集准确率：62.00%   测试集准确率：63.38%
当前epoch：5 [25000/50000]   损失：1.126291  训练集准确率：61.98%   测试集准确率：64.27%
当前epoch：6 [0/50000]   损失：0.957984  训练集准确率：64.20%   测试集准确率：64.77%
当前epoch：6 [25000/50000]   损失：1.099079  训练集准确率：64.16%   测试集准确率：66.13%
当前epoch：7 [0/50000]   损失：0.914080  训练集准确率：67.40%   测试集准确率：64.7

In [33]:
loss_ = pd.DataFrame()
ac_ = pd.DataFrame()

In [93]:
loss_['无dropout'] = train_loss
ac_['无dropout'] = test_right
loss_.to_csv('loss_.csv')
ac_.to_csv('ac_.csv')

In [66]:
ac_

,relu,sigmoid,tanh,leakyrelu,每次池化前三次卷积,每次池化前两次卷积,每次池化前一次卷积,5*5卷积核,3*3卷积核,7*7卷积核,两层fc
0,13.62,9.58,10.08,10.099999,9.82,9.78,16.55,16.55,11.139999,8.99,13.62
1,89.52,11.349999,90.61,90.77,91.61,82.04,88.32,88.32,85.11,87.88,89.52
2,93.0,58.609997,94.759995,94.439995,93.92,90.619995,93.13,93.13,91.439995,91.35,93.0
3,95.17,79.86,96.04,96.04,95.38,95.03,94.81,94.81,93.829994,94.119995,95.17
4,96.27,87.299995,96.829994,96.64,96.54,95.649994,95.009995,95.009995,94.99,95.409996,96.27
5,96.7,90.329994,97.53,97.189995,97.14,95.579994,96.45,96.45,96.04,95.77,96.7
6,97.03,91.85,97.74,97.22,97.1,96.829994,97.11,97.11,96.57,96.78,97.03
7,97.47,92.93,97.89,97.39,97.57,97.189995,96.93,96.93,96.689995,97.14,97.47
8,97.68,93.77,98.03,97.99,97.7,97.52,97.31,97.31,96.17,97.28,97.68
9,97.619995,94.46,97.84,97.84,97.729996,97.659996,97.47,97.47,97.22,97.759995,97.619995
